<a href="https://colab.research.google.com/github/manueljosema/Primer-commit-a-adir-README.md/blob/main/Pandas_para_Ciencia_de_Datos_II_Proyecto_I_Parte_III_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files
uploaded = files.upload()

Saving retail_sales_dataset.csv to retail_sales_dataset.csv


### 1) Transformación de Datos
* Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.
*Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).

In [6]:
# Cargar datos de un archivo CSV
df = pd.read_csv('/content/retail_sales_dataset.csv')

# Mostrar las primeras 5 filas del da
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [17]:

# Example custom function: Calculate the average price per unit for each product category.
def average_price_per_category(df):
    """
    Calculates the average price per unit for each product category.

    Args:
        df: Input DataFrame.

    Returns:
        pandas.Series: A Series containing the average price per unit for each product category.
    """
    return df.groupby('Product Category')['Price per Unit'].mean()

# Example custom function: Identify high-value customers (Total Amount > 1000).
def identify_high_value_customers(df):
    """
    Identifies high-value customers based on total amount spent.

    Args:
        df: Input DataFrame.

    Returns:
        pandas.DataFrame: A DataFrame containing only high-value customer transactions.
    """

    high_value_customers = df[df['Total Amount'] > 1000]
    return high_value_customers


# Apply the custom functions
avg_prices = average_price_per_category(df)
print("Average price per category:\n", avg_prices)


high_value = identify_high_value_customers(df)
print("\nHigh-value customers:\n",high_value)

Average Prices per Category:
 Product Category
Beauty         184.055375
Clothing       174.287749
Electronics    181.900585
Name: Price per Unit, dtype: float64

High Value Transactions:
      Transaction ID       Date Customer ID  Gender  Age Product Category  \
12               13 2023-08-05     CUST013    Male   22      Electronics   
14               15 2023-01-16     CUST015  Female   42      Electronics   
15               16 2023-02-17     CUST016    Male   19         Clothing   
30               31 2023-05-23     CUST031    Male   44      Electronics   
45               46 2023-06-26     CUST046  Female   20      Electronics   
..              ...        ...         ...     ...  ...              ...   
941             942 2023-03-18     CUST942    Male   51         Clothing   
942             943 2023-10-16     CUST943  Female   57         Clothing   
945             946 2023-05-08     CUST946    Male   62      Electronics   
955             956 2023-08-19     CUST956    Male 

In [7]:
# Crear una columna con el ingreso total por venta
df['Total Amount'] = df['Price per Unit'] * df['Quantity']

# Normalizar las ventas
df['Normalized Sales'] = df['Total Amount'] / df['Total Amount'].max()

# Mostrar el dataframe con las nuevas columnas
print(df.head())

   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  2023-11-24     CUST001    Male   34           Beauty   
1               2  2023-02-27     CUST002  Female   26         Clothing   
2               3  2023-01-13     CUST003    Male   50      Electronics   
3               4  2023-05-21     CUST004    Male   37         Clothing   
4               5  2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  Normalized Sales  
0         3              50           150             0.075  
1         2             500          1000             0.500  
2         1              30            30             0.015  
3         1             500           500             0.250  
4         2              50           100             0.050  


In [10]:
# Calcular cuantiles para la categorización
quantiles = df['Total Amount'].quantile([0.25, 0.75])

# Definir una función para categorizar ventas.
def categorize_sales(amount):
    if amount <= quantiles[0.25]:
        return 'Baja'
    elif amount <= quantiles[0.75]:
        return 'Media'
    else:
        return 'Alta'

# Aplica la función de categorización para crear una nueva columna
df['Sales Category'] = df['Total Amount'].apply(categorize_sales)

# Mostrar el DataFrame con la nueva columna (opcional)
print(df.head())

   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  2023-11-24     CUST001    Male   34           Beauty   
1               2  2023-02-27     CUST002  Female   26         Clothing   
2               3  2023-01-13     CUST003    Male   50      Electronics   
3               4  2023-05-21     CUST004    Male   37         Clothing   
4               5  2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  Normalized Sales Sales Category  
0         3              50           150             0.075          Media  
1         2             500          1000             0.500           Alta  
2         1              30            30             0.015           Baja  
3         1             500           500             0.250          Media  
4         2              50           100             0.050          Media  


### 3) Agrupación y Agregación
* Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.
* Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.

In [13]:
# Agrupar por 'Product Category' y 'Sales Category' y calcular la suma de 'Total Amount'
product_sales_category_group = df.groupby(['Product Category', 'Sales Category'])['Total Amount'].sum()
print("Grouped by Product Category and Sales Category:\n", product_sales_category_group)

# Convierte la columna 'Date' a objetos de fecha y hora.
df['Date'] = pd.to_datetime(df['Date'])

# Extrae el mes de la columna 'Fecha'.
df['Month'] = df['Date'].dt.month

# Agrupar por 'Product Category' y 'Month' y calcular la media de 'Total Amount'
product_month_group = df.groupby(['Product Category', 'Month'])['Total Amount'].mean()
print("\nGrouped by Product Category and Month:\n", product_month_group)

# Agrupar por 'Gender' y 'Age' y calcular el conteo de transacciones.
gender_age_group = df.groupby(['Gender', 'Age'])['Transaction ID'].count()
print("\nGrouped by Gender and Age:\n", gender_age_group)

Grouped by Product Category and Sales Category:
 Product Category  Sales Category
Beauty            Alta               94300
                  Baja                3060
                  Media              46155
Clothing          Alta               92500
                  Baja                3885
                  Media              59195
Electronics       Alta              100000
                  Baja                4310
                  Media              52595
Name: Total Amount, dtype: int64

Grouped by Product Category and Month:
 Product Category  Month
Beauty            1        535.769231
                  2        539.807692
                  3        502.142857
                  4        410.517241
                  5        444.642857
                  6        439.800000
                  7        595.925926
                  8        407.916667
                  9        316.000000
                  10       495.322581
                  11       388.000000
               

In [16]:
# Agrupar datos por 'Categoría de producto' y aplicar funciones de agregación.
aggregated_df = df.groupby('Product Category').agg({
    'Quantity': ['sum', 'mean', 'count', 'min', 'max', 'std', 'var'],
    'Price per Unit': ['sum', 'mean', 'count', 'min', 'max', 'std', 'var'],
    'Total Amount': ['sum', 'mean', 'count', 'min', 'max', 'std', 'var']
})

# Mostrar los resultados agregados
aggregated_df

Quantity                                              \
                      sum      mean count min max       std       var   
Product Category                                                        
Beauty                771  2.511401   307   1   4  1.100668  1.211471   
Clothing              894  2.547009   351   1   4  1.142647  1.305641   
Electronics           849  2.482456   342   1   4  1.153084  1.329603   

                 Price per Unit                    ...                   \
                            sum        mean count  ...  max         std   
Product Category                                   ...                    
Beauty                    56505  184.055375   307  ...  500  194.929399   
Clothing                  61175  174.287749   351  ...  500  186.017065   
Electronics               62210  181.900585   342  ...  500  189.049673   

                               Total Amount                              \
                           var          sum        mean count min   max   
Product Category                                                          
Beauty            37997.470780       143515  467.475570   307  25  2000   
Clothing          34602.348392       155580  443.247863   351  25  2000   
Electronics       35739.778944       156905  458.786550   342  25  2000   

                                             
                         std            var  
Product Category                             
Beauty            563.612788  317659.374401  
Clothing          550.695917  303265.992674  
Electronics       567.540150  322101.822341  

[3 rows x 21 columns]

### 4) Análisis Personalizado con apply
* Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.
* Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.

In [18]:
def average_price_per_category(df):
    """
    Calculates the average price per unit for each product category.

    Args:
        df: Input DataFrame.

    Returns:
        pandas.Series: A Series containing the average price per unit for each product category.
    """
    return df.groupby('Product Category')['Price per Unit'].mean()

# Example custom function: Identify high-value customers (Total Amount > 1000).
def identify_high_value_customers(df):
    """
    Identifies high-value customers based on total amount spent.

    Args:
        df: Input DataFrame.

    Returns:
        pandas.DataFrame: A DataFrame containing only high-value customer transactions.
    """

    high_value_customers = df[df['Total Amount'] > 1000]
    return high_value_customers


# Apply the custom functions
avg_prices = average_price_per_category(df)
print("Average price per category:\n", avg_prices)


high_value = identify_high_value_customers(df)
print("\nHigh-value customers:\n",high_value)

Average price per category:
 Product Category
Beauty         184.055375
Clothing       174.287749
Electronics    181.900585
Name: Price per Unit, dtype: float64

High-value customers:
      Transaction ID       Date Customer ID  Gender  Age Product Category  \
12               13 2023-08-05     CUST013    Male   22      Electronics   
14               15 2023-01-16     CUST015  Female   42      Electronics   
15               16 2023-02-17     CUST016    Male   19         Clothing   
30               31 2023-05-23     CUST031    Male   44      Electronics   
45               46 2023-06-26     CUST046  Female   20      Electronics   
..              ...        ...         ...     ...  ...              ...   
941             942 2023-03-18     CUST942    Male   51         Clothing   
942             943 2023-10-16     CUST943  Female   57         Clothing   
945             946 2023-05-08     CUST946    Male   62      Electronics   
955             956 2023-08-19     CUST956    Male   30

### 5) Documentación
* Comentarios claros: Documenta claramente cada paso del análisis, explicando qué se hizo y por qué se hizo.
* Código legible: Asegúrate de que el código sea legible y esté bien comentado.

In [ ]:
# Este código realiza la transformación y el análisis de datos del dataset de ventas.

# 1. Cargar los datos del archivo CSV.
# 2. Crear dos nuevas columnas: 'Ingreso Total' y 'Ventas Normalizadas'.
# 3. Clasificar las ventas en categorías: 'Alta', 'Media', 'Baja'.
# 4. Agrupar los datos por:
#    - Producto y tienda.
#    - Producto y mes.
# 5. Aplicar funciones de agregación para obtener estadísticas descriptivas de cada grupo.
# 6. Crear una función personalizada para calcular la desviación de cada venta respecto a la media del grupo.
# 7. Aplicar la función personalizada a cada grupo.